# Executable example of how to run the Surgeon

### Copyright (c) 2021 Schiessler et al.

Note that running this file will download a large amount of data. Running the Surgeon potentially requires a 
large amount of computational resources. If you wish to limit the runtime, modify hyperparameters in the 
beginning of the NAS.py file accordingly.

### Required imports

In [1]:
# 3rd party packages
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

# Surgeon specific packages
import NAS

### Data loading and preprocessing

In [2]:
seed = 97

In [3]:
dataset_name = 'eurosat/rgb'
split = ['train[:80%]', 'train[80%:]']

In [4]:
data_dir = '.'

In [5]:
(ds_train, ds_valid), ds_info = tfds.load(dataset_name,
                                     data_dir=data_dir,
                                     as_supervised=True,
                                     shuffle_files=True,
                                     with_info=True,
                                     split=split,
                                     read_config=tfds.ReadConfig(shuffle_seed=seed)) 

In [6]:
def preprocess(ds, seed, normalize_fn, shuffle=False, augment_fn=None):
    ds = ds.map(normalize_fn, num_parallel_calls=tf.data.experimental.AUTOTUNE)

    if shuffle:
        ds = ds.cache()
        if augment_fn is not None:
            ds = ds.map(augment_fn, num_parallel_calls=tf.data.experimental.AUTOTUNE)
        ds = ds.shuffle(1000, seed=seed)
        ds = ds.batch(32)
        if augment_fn is not None:
            ds = ds.repeat(2)
    else:
        ds = ds.batch(32)
        ds = ds.cache()

    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)

    return ds

In [7]:
def normalize(image, label):
    return tf.cast(image, tf.float32)/255., label

In [8]:
def augment(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    return image, label

In [9]:
# Perform preprocessing
ds_train = preprocess(ds_train, seed, normalize, shuffle=True, augment_fn=None)
ds_valid = preprocess(ds_valid, seed, normalize, shuffle=False, augment_fn=None)

### Generate initial model

In [10]:
input_shape = ds_info.features['image'].shape

In [11]:
num_classes = ds_info.features['label'].num_classes

In [12]:
hidden_layer_sizes = [10, 10, 10]

In [13]:
model = keras.models.Sequential([keras.layers.Flatten(input_shape=input_shape)])

for size in hidden_layer_sizes:
    model.add(keras.layers.Dense(size, activation='relu'))
    
model.add(keras.layers.Dense(num_classes, activation='softmax'))

In [14]:
compiler_information={
    'loss': keras.losses.sparse_categorical_crossentropy,
    'optimizer': keras.optimizers.SGD(learning_rate=0.005),
    'metrics':['accuracy']
}

In [15]:
model.compile(loss=compiler_information['loss'],
              optimizer=compiler_information['optimizer'],
              metrics=compiler_information['metrics'])

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 12288)             0         
_________________________________________________________________
dense (Dense)                (None, 10)                122890    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_3 (Dense)              (None, 10)                110       
Total params: 123,220
Trainable params: 123,220
Non-trainable params: 0
_________________________________________________________________


### Prepare and run the Surgeon

In [17]:
data = (ds_train, ds_valid)

In [18]:
nas = NAS.NasTrainer(data, compiler_information, scoring_criterion='val_accuracy')

In [19]:
optimized_modification = nas.run(model, random_only=False, verbose=2)

Started run
Initial training
Current epoch: 11 to 20
  19 candidates created
..try 1
  19 candidates found
  7 modifications selected


  7 modifications allowed
  1 modifications kept in current branches
Current epoch: 21 to 30
  19 candidates created
..try 1
  19 candidates found
  7 modifications selected


  7 modifications allowed
  2 modifications kept in current branches
Current epoch: 31 to 40
  19 candidates created
  25 candidates created
..try 1
  44 candidates found
  8 modifications selected


  8 modifications allowed
  1 modifications kept in current branches
Current epoch: 41 to 50
  25 candidates created
..try 1
  25 candidates found
  7 modifications selected


  7 modifications allowed
  2 modifications kept in current branches
Current epoch: 51 to 60
  25 candidates created
  31 candidates created
..try 1
  56 candidates found
  8 modifications selected


  8 modifications allowed
  1 modifications kept in current branches
Current epoch: 61 to 70
  25 candidates created
..try 1
  25 candidates found
  7 modifications selected


  7 modifications allowed
..try 2
  18 candidates found
  6 modifications selected
  6 modifications allowed
  1 modifications kept in current branches
Current epoch: 71 to 80
  31 candidates created
..try 1
  31 candidates found
  7 modifications selected


  7 modifications allowed
  1 modifications kept in current branches
Current epoch: 81 to 90
  37 candidates created
..try 1
  37 candidates found
  7 modifications selected


  7 modifications allowed
  1 modifications kept in current branches
Current epoch: 91 to 100
  43 candidates created
..try 1
  43 candidates found
  7 modifications selected


  7 modifications allowed
  1 modifications kept in current branches
  1 modifications allowed


In [20]:
# verbose parameter controls how much information the Surgeon displays. Note that even when set to 0,
# some tensorflow warnings will still be triggered and can't be suppressed, which is due to a bug in
# the current tensorflow package

### Information that can be accessed after running the Surgeon

(examples, see Modification.py for all details)

In [21]:
type(optimized_modification)

Modification.Modification

In [22]:
type(optimized_modification.model)

tensorflow.python.keras.engine.sequential.Sequential

In [23]:
optimized_modification.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 12288)             0         
_________________________________________________________________
dense (Dense)                (None, 10)                122890    
_________________________________________________________________
dense_1 (Dense)              (None, 12)                132       
_________________________________________________________________
dense_4 (Dense)              (None, 27)                324       
_________________________________________________________________
dense_5 (Dense)              (None, 7)                 189       
_________________________________________________________________
dense_6 (Dense)              (None, 7)                 56        
_________________________________________________________________
dense_7 (Dense)              (None, 7)                 5

In [24]:
optimized_modification.description

'B ~ EP 11 AN2 dense_1** ~ EP 21 T3 dense_2 ~ EP 31 B ~ EP 41 AN20 dense_4 ~ EP 51 B ~ EP 61 T3 dense_2 ~ EP 71 AL dense_5 ~ EP 81 AL dense_6 ~ EP 91 AN2 dense_2**'

In [25]:
optimized_modification.history['val_accuracy']

[0.2124074101448059,
 0.27759259939193726,
 0.3127777874469757,
 0.3253703713417053,
 0.35092592239379883,
 0.3453703820705414,
 0.3190740644931793,
 0.30259260535240173,
 0.37962964177131653,
 0.3853703737258911,
 0.378888875246048,
 0.3570370376110077,
 0.385740727186203,
 0.3866666555404663,
 0.4125925898551941,
 0.38740742206573486,
 0.419259250164032,
 0.41499999165534973,
 0.43037036061286926,
 0.44296297430992126,
 0.45629629492759705,
 0.4501851797103882,
 0.465925931930542,
 0.4098148047924042,
 0.4005555510520935,
 0.4653703570365906,
 0.443148136138916,
 0.44203704595565796,
 0.46666666865348816,
 0.4605555534362793,
 0.482962965965271,
 0.47203704714775085,
 0.47648146748542786,
 0.4770370423793793,
 0.503333330154419,
 0.3829629719257355,
 0.46000000834465027,
 0.48462963104248047,
 0.4940740764141083,
 0.4964814782142639,
 0.4905555546283722,
 0.502407431602478,
 0.489259272813797,
 0.5135185122489929,
 0.4861111044883728,
 0.46666666865348816,
 0.49259260296821594,
 0.46